In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Load your dataset (adjust the path as needed)
train_df = pd.read_csv("train.csv")

# Set seaborn style
sns.set(style="whitegrid")

# --------- Plot 1: Histogram of Age ---------
plt.figure(figsize=(8, 5))
sns.histplot(train_df['age'], bins=20, kde=True, color='skyblue')
plt.title("Age Distribution")
plt.xlabel("Age")
plt.ylabel("Count")
plt.tight_layout()
plt.show()

# --------- Plot 2: Bar Chart of Gender ---------
plt.figure(figsize=(6, 4))
sns.countplot(x='gender', data=train_df, palette='pastel')
plt.title("Gender Distribution")
plt.xlabel("Gender")
plt.ylabel("Count")
plt.tight_layout()
plt.show()

# --------- Plot 3: Bar Chart of Ethnicity ---------
plt.figure(figsize=(10, 6))
sns.countplot(y='ethnicity', data=train_df, 
              order=train_df['ethnicity'].value_counts().index,
              palette='muted')
plt.title("Ethnicity Distribution")
plt.xlabel("Count")
plt.ylabel("Ethnicity")
plt.tight_layout()
plt.show()


ModuleNotFoundError: No module named 'pandas'